In [1]:
import pandas

import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

Definisanje globalnih promenljivih za lakse menjanje

In [2]:
PUTANJA = 'C:/Users/User/Desktop/AI'
KATEGORIJE = ['ZDRAV', 'VIRUS', 'BAKTERIJA']
IMG_SIZE = 400

In [3]:
from tensorflow.keras.callbacks import TensorBoard
import time

IME = "Model-Pluca-9x2-{}-v2_0".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(IME))

Pravljenje niza labela "train_y" i punimo ga brojevnim reprezentacijama zadatih kategorija

In [4]:
csv_fajl = pandas.read_csv(PUTANJA + r"\chest_xray_metadata.csv")  # promeniti na svoju lokaciju

# uklanjamo pusace zbog manjka slika (takodje je tako navedeno u zadatku)

train_y = np.ndarray(0)

for index, row in csv_fajl.iterrows():
    if row['Label'] != 'Normal':
        if row['Label_1_Virus_category'] == 'Virus':
            train_y = np.append(train_y, 1)
        elif row['Label_1_Virus_category'] == 'bacteria':
            train_y = np.append(train_y, 2)
        else:
            print("Neodgovarajuca labela na slici {}".format(row['X_ray_image_name']))
    else:
        train_y = np.append(train_y, 0)
        
trainy2 = np.ndarray(0)

for index, row in csv_fajl.iterrows():
    if row['Label'] != 'Normal':
        if row['Label_1_Virus_category'] == 'Virus':
            trainy2 = np.append(trainy2, 1)
        elif row['Label_1_Virus_category'] == 'bacteria':
            trainy2 = np.append(trainy2, 2)
        else:
            trainy2 = np.append(trainy2, -1)
    else:
        trainy2 = np.append(trainy2, 0)

Neodgovarajuca labela na slici acute-respiratory-distress-syndrome-ards.jpg
Neodgovarajuca labela na slici acute-respiratory-distress-syndrome-ards-1.jpg


In [5]:
csv_fajl.head()
training_data = []

for index, row in csv_fajl.iterrows():
    if trainy2[index] != -1:
        img_name = row['X_ray_image_name']
        lokacija = PUTANJA + '/snimci/' + str(img_name)
        try:# za slucaj da neka slika ostecena 
            slika = cv2.imread(lokacija, cv2.IMREAD_GRAYSCALE)
            slika = cv2.resize(slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
            training_data.append([slika, int(trainy2[index])])
        except:
            print("Ostecena slika " + str(img_name))

In [6]:
print(len(training_data))

5284


In [7]:
import random

random.shuffle(training_data)

In [8]:
X = []
y = []

In [9]:
for features, label in training_data:
    X.append(features)
    y.append(label)
    
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [10]:
import pickle

pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

Kreiranje CNN-a

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

In [12]:
import pickle
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

X = X/255.0

In [ ]:
#import os
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Trenutni META
####################### 
from tensorflow.keras.optimizers import RMSprop
model = tf.keras.models.Sequential([
  
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    # model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
    tf.keras.layers.Flatten(input_shape = X.shape[1:]),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(7)
])

####### to get the summary of the model
model.summary()

####### configure the model for traning by adding metrics
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy']
             )
#######################

# Ne radi
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss="categorical_crossentropy",
             optimizer="adam",
             metrics=['accuracy'])

In [14]:
from tensorflow.keras.optimizers import RMSprop

dense_layers = [0]
layer_sizes = [32]
conv_layers = [1]

for epp in [20]:#[1, 2, 3, 5, 10]:
    for dense_layer in dense_layers:
        for layer_size in layer_sizes:
            for conv_layer in conv_layers:
                NAME = "{}-dense---{}-nodes---{}-conv---{}-v2_0".format(dense_layer, layer_size, conv_layer, int(time.time()))
                tensorboard = TensorBoard(log_dir='logs_test/{}'.format(IME))
                print(NAME)

                model = tf.keras.models.Sequential()

                model.add(Conv2D(layer_size, (3, 3), input_shape = X.shape[1:]))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

                for l in range(conv_layer-1):
                    model.add(Conv2D(layer_size, (3, 3)))
                    model.add(Activation('relu'))
                    model.add(MaxPooling2D(pool_size=(2, 2)))

                model.add(Flatten())

                for l in range(dense_layer):
                    model.add(Dense(layer_size))
                    model.add(Activation('relu'))


                model.add(Dense(7))

                model.compile(optimizer='adam',
                              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                              metrics=['accuracy']
                )
                model.fit(np.array(X), np.array(y), batch_size=9,  epochs=epp, validation_split=0.1, callbacks=[tensorboard])
                model.save('saved_model\\100ep' + str(epp))

0-dense---32-nodes---1-conv---1598275451-v2_0
Epoch 1/20
529/529 [==============================] - 149s 282ms/step - loss: 1.1420 - accuracy: 0.7310 - val_loss: 0.4641 - val_accuracy: 0.7996
Epoch 2/20
529/529 [==============================] - 141s 267ms/step - loss: 0.3750 - accuracy: 0.8475 - val_loss: 0.5085 - val_accuracy: 0.7788
Epoch 3/20
529/529 [==============================] - 211s 399ms/step - loss: 0.2830 - accuracy: 0.8923 - val_loss: 0.6154 - val_accuracy: 0.7467
Epoch 4/20
529/529 [==============================] - 141s 266ms/step - loss: 0.2053 - accuracy: 0.9209 - val_loss: 0.5465 - val_accuracy: 0.7977
Epoch 5/20
529/529 [==============================] - 141s 266ms/step - loss: 0.1176 - accuracy: 0.9611 - val_loss: 0.6345 - val_accuracy: 0.7807
Epoch 6/20
529/529 [==============================] - 143s 270ms/step - loss: 0.0680 - accuracy: 0.9840 - val_loss: 0.6329 - val_accuracy: 0.7864
Epoch 7/20
529/529 [==============================] - 144s 272ms/step - loss: 

0-dense---32-nodes---1-conv---1598266784-v2_0
  2/529 [..............................] - ETA: 14:29 - loss: 1.3043 - accuracy: 0.3889WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.788921). Check your callbacks.
  3/529 [..............................] - ETA: 10:40 - loss: 15.5834 - accuracy: 0.3704WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.195120). Check your callbacks.
  4/529 [..............................] - ETA: 8:35 - loss: 21.4312 - accuracy: 0.3889 WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.598558). Check your callbacks.
529/529 [==============================] - 228s 430ms/step - loss: 1.1297 - accuracy: 0.7354 - val_loss: 0.4790 - val_accuracy: 0.7921
WARNING:tensorflow:From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model\100ep1\assets
0-dense---32-nodes---1-conv---1598267581-v2_0
Epoch 1/2
  2/529 [..............................] - ETA: 8:33 - loss: 7.7968 - accuracy: 0.3333   WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.863747). Check your callbacks.
  3/529 [..............................] - ETA: 6:43 - loss: 11.3669 - accuracy: 0.3333WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.325281). Check your callbacks.
529/529 [==============================] - 148s 280ms/step - loss: 1.1303 - accuracy: 0.7293 - val_loss: 0.4637 - val_accuracy: 0.7977
Epoch 2/2
529/529 [==============================] - 141s 266ms/step - loss: 0.4004 - accuracy: 0.8301 - val_loss: 0.4663 - val_accuracy: 0.8091
INFO:tensorflow:Assets written to: saved_model\100ep2\assets
0-dense---32-nodes---1-conv---1598269335-v2_0
Epoch 1/3
  2/529 [..............................] - ETA: 41:52 - loss: 2.2307 - accuracy: 0.2778 WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (4.423901). Check your callbacks.
  3/529 [..............................] - ETA: 28:58 - loss: 5.1164 - accuracy: 0.2593WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.564261). Check your callbacks.
  4/529 [..............................] - ETA: 22:16 - loss: 12.7376 - accuracy: 0.2222WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.782630). Check your callbacks.
529/529 [==============================] - 155s 294ms/step - loss: 0.9405 - accuracy: 0.7348 - val_loss: 0.4552 - val_accuracy: 0.7977
Epoch 2/3
529/529 [==============================] - 140s 264ms/step - loss: 0.3349 - accuracy: 0.8675 - val_loss: 0.4919 - val_accuracy: 0.7958
Epoch 3/3
529/529 [==============================] - 139s 262ms/step - loss: 0.2237 - accuracy: 0.9121 - val_loss: 0.5870 - val_accuracy: 0.7694
INFO:tensorflow:Assets written to: saved_model\100ep3\assets
0-dense---32-nodes---1-conv---1598271457-v2_0
Epoch 1/5
  2/529 [..............................] - ETA: 10:34 - loss: 3.9059 - accuracy: 0.3333WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.442021). Check your callbacks.
  3/529 [..............................] - ETA: 8:24 - loss: 9.9035 - accuracy: 0.3333 WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.979047). Check your callbacks.
  4/529 [..............................] - ETA: 7:02 - loss: 9.4066 - accuracy: 0.3889WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.490022). Check your callbacks.
529/529 [==============================] - 146s 275ms/step - loss: 0.8968 - accuracy: 0.7312 - val_loss: 0.5672 - val_accuracy: 0.7543
Epoch 2/5
529/529 [==============================] - 139s 263ms/step - loss: 0.3981 - accuracy: 0.8303 - val_loss: 0.5161 - val_accuracy: 0.7845
Epoch 3/5
529/529 [==============================] - 139s 264ms/step - loss: 0.2589 - accuracy: 0.8997 - val_loss: 0.5443 - val_accuracy: 0.7694
Epoch 4/5
529/529 [==============================] - 139s 263ms/step - loss: 0.1555 - accuracy: 0.9468 - val_loss: 0.5800 - val_accuracy: 0.7996
Epoch 5/5
529/529 [==============================] - 139s 263ms/step - loss: 0.1123 - accuracy: 0.9661 - val_loss: 0.6508 - val_accuracy: 0.7921
INFO:tensorflow:Assets written to: saved_model\100ep5\assets
0-dense---32-nodes---1-conv---1598272767-v2_0
Epoch 1/10
  2/529 [..............................] - ETA: 27:19 - loss: 1.6887 - accuracy: 0.2778WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (3.777326). Check your callbacks.
  3/529 [..............................] - ETA: 20:15 - loss: 10.1539 - accuracy: 0.2963WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (2.002109). Check your callbacks.
  4/529 [..............................] - ETA: 15:44 - loss: 17.3437 - accuracy: 0.2778WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.001553). Check your callbacks.
529/529 [==============================] - 152s 287ms/step - loss: 0.9434 - accuracy: 0.7392 - val_loss: 0.4837 - val_accuracy: 0.8034
Epoch 2/10
529/529 [==============================] - 140s 265ms/step - loss: 0.3567 - accuracy: 0.8538 - val_loss: 0.5434 - val_accuracy: 0.7713
Epoch 3/10
529/529 [==============================] - 140s 264ms/step - loss: 0.2203 - accuracy: 0.9213 - val_loss: 0.5178 - val_accuracy: 0.8015
Epoch 4/10
529/529 [==============================] - 139s 263ms/step - loss: 0.1258 - accuracy: 0.9603 - val_loss: 0.5930 - val_accuracy: 0.7958
Epoch 5/10
529/529 [==============================] - 140s 264ms/step - loss: 0.0672 - accuracy: 0.9836 - val_loss: 0.7252 - val_accuracy: 0.7921
Epoch 6/10
529/529 [==============================] - 140s 264ms/step - loss: 0.0519 - accuracy: 0.9893 - val_loss: 0.7481 - val_accuracy: 0.7732
Epoch 7/10
529/529 [==============================] - 140s 265ms/step - loss: 0.0190 - accuracy: 0.9983 - val_loss: 0.8373 - val_accuracy: 0.7826
Epoch 8/10
529/529 [==============================] - 141s 266ms/step - loss: 0.0118 - accuracy: 0.9996 - val_loss: 0.9323 - val_accuracy: 0.7826
Epoch 9/10
529/529 [==============================] - 140s 265ms/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.9823 - val_accuracy: 0.7940
Epoch 10/10
529/529 [==============================] - 141s 267ms/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 1.0245 - val_accuracy: 0.7921
INFO:tensorflow:Assets written to: saved_model\100ep10\assets

0-dense---32-nodes---1-conv---1598275451-v2_0
Epoch 1/20
  2/529 [..............................] - ETA: 17:06 - loss: 29.8534 - accuracy: 0.1667WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (2.019790). Check your callbacks.
  3/529 [..............................] - ETA: 13:12 - loss: 23.7078 - accuracy: 0.3333WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.007780). Check your callbacks.
  4/529 [..............................] - ETA: 10:32 - loss: 27.0637 - accuracy: 0.3056WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.504409). Check your callbacks.
529/529 [==============================] - 149s 282ms/step - loss: 1.1420 - accuracy: 0.7310 - val_loss: 0.4641 - val_accuracy: 0.7996
Epoch 2/20
529/529 [==============================] - 141s 267ms/step - loss: 0.3750 - accuracy: 0.8475 - val_loss: 0.5085 - val_accuracy: 0.7788
Epoch 3/20
529/529 [==============================] - 211s 399ms/step - loss: 0.2830 - accuracy: 0.8923 - val_loss: 0.6154 - val_accuracy: 0.7467
Epoch 4/20
529/529 [==============================] - 141s 266ms/step - loss: 0.2053 - accuracy: 0.9209 - val_loss: 0.5465 - val_accuracy: 0.7977
Epoch 5/20
529/529 [==============================] - 141s 266ms/step - loss: 0.1176 - accuracy: 0.9611 - val_loss: 0.6345 - val_accuracy: 0.7807
Epoch 6/20
529/529 [==============================] - 143s 270ms/step - loss: 0.0680 - accuracy: 0.9840 - val_loss: 0.6329 - val_accuracy: 0.7864
Epoch 7/20
529/529 [==============================] - 144s 272ms/step - loss: 0.0382 - accuracy: 0.9935 - val_loss: 0.8607 - val_accuracy: 0.7732
Epoch 8/20
529/529 [==============================] - 140s 264ms/step - loss: 0.0242 - accuracy: 0.9975 - val_loss: 0.7493 - val_accuracy: 0.7902
Epoch 9/20
529/529 [==============================] - 140s 265ms/step - loss: 0.0116 - accuracy: 0.9992 - val_loss: 0.8219 - val_accuracy: 0.7769
Epoch 10/20
529/529 [==============================] - 141s 266ms/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.8806 - val_accuracy: 0.7958
Epoch 11/20
529/529 [==============================] - 141s 267ms/step - loss: 0.0032 - accuracy: 1.0000 - val_loss: 0.9214 - val_accuracy: 0.7883
Epoch 12/20
529/529 [==============================] - 141s 267ms/step - loss: 0.0021 - accuracy: 1.0000 - val_loss: 0.9256 - val_accuracy: 0.8110
Epoch 13/20
529/529 [==============================] - 142s 268ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 1.0240 - val_accuracy: 0.7713
Epoch 14/20
529/529 [==============================] - 142s 269ms/step - loss: 8.1678e-04 - accuracy: 1.0000 - val_loss: 1.0636 - val_accuracy: 0.7977
Epoch 15/20
529/529 [==============================] - 142s 269ms/step - loss: 5.8631e-04 - accuracy: 1.0000 - val_loss: 1.0640 - val_accuracy: 0.7788
Epoch 16/20
529/529 [==============================] - 144s 272ms/step - loss: 3.6192e-04 - accuracy: 1.0000 - val_loss: 1.1109 - val_accuracy: 0.7807
Epoch 17/20
529/529 [==============================] - 143s 270ms/step - loss: 2.5620e-04 - accuracy: 1.0000 - val_loss: 1.1257 - val_accuracy: 0.7883
Epoch 18/20
529/529 [==============================] - 143s 270ms/step - loss: 1.8450e-04 - accuracy: 1.0000 - val_loss: 1.1571 - val_accuracy: 0.7921
Epoch 19/20
529/529 [==============================] - 143s 270ms/step - loss: 1.5038e-04 - accuracy: 1.0000 - val_loss: 1.1794 - val_accuracy: 0.7883
Epoch 20/20
529/529 [==============================] - 143s 270ms/step - loss: 1.0597e-04 - accuracy: 1.0000 - val_loss: 1.2167 - val_accuracy: 0.7826
INFO:tensorflow:Assets written to: saved_model\100ep20\assets

# model.fit(np.array(X), np.array(y), batch_size=9,  epochs=2, validation_split=0.1, callbacks=[tensorboard])

In [ ]:
#model.fit(np.array(X), np.array(y), batch_size=9,  epochs=5, validation_split=0.1, callbacks=[tensorboard])
X = []
y = []

In [ ]:
#model.save('saved_model\\100')

za 400:
    Epoch 1/10
    529/529 518s 979ms/step - loss: 1.5214e-07 - accuracy: 0.2466 - val_loss: 1.3859e-07 - val_accuracy: 0.3119
    Epoch 2/10
    529/529 507s 958ms/step - loss: 1.5214e-07 - accuracy: 0.2466 - val_loss: 1.3859e-07 - val_accuracy: 0.3119
    
za 350:
    Epoch 1/10
    529/529 394s 745ms/step - loss: 1.4998e-07 - accuracy: 0.2527 - val_loss: 1.5797e-07 - val_accuracy: 0.2212

In [ ]:
''' #conv i dense su zamenjeni
0-conv---32-nodes---1-dense---1595918749
Epoch 1/2
  2/529 [..............................] - ETA: 5:37 - loss: 13.7277 - accuracy: 0.2778WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.579315). Check your callbacks.
529/529 [==============================] - 78s 148ms/step - loss: 1.1279 - accuracy: 0.7305 - val_loss: 0.4849 - val_accuracy: 0.7958
Epoch 2/2
529/529 [==============================] - 77s 145ms/step - loss: 0.3876 - accuracy: 0.8400 - val_loss: 0.5005 - val_accuracy: 0.7788
0-conv---32-nodes---2-dense---1595918911
Epoch 1/2
  2/529 [..............................] - ETA: 6:36 - loss: 2.6182 - accuracy: 0.2222  WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.639454). Check your callbacks.
529/529 [==============================] - 130s 247ms/step - loss: 0.6906 - accuracy: 0.7219 - val_loss: 0.5290 - val_accuracy: 0.7543
Epoch 2/2
529/529 [==============================] - 127s 240ms/step - loss: 0.4883 - accuracy: 0.7965 - val_loss: 0.5098 - val_accuracy: 0.7750
0-conv---32-nodes---3-dense---1595919185
Epoch 1/2
  2/529 [..............................] - ETA: 28:20 - loss: 1.6812 - accuracy: 0.2222 WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (3.884267). Check your callbacks.
  3/529 [..............................] - ETA: 19:36 - loss: 1.8325 - accuracy: 0.2222WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (3.170902). Check your callbacks.
  4/529 [..............................] - ETA: 15:14 - loss: 1.7329 - accuracy: 0.3056WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.585950). Check your callbacks.
529/529 [==============================] - 149s 282ms/step - loss: 0.6922 - accuracy: 0.7072 - val_loss: 0.5050 - val_accuracy: 0.7826
Epoch 2/2
529/529 [==============================] - 138s 262ms/step - loss: 0.4884 - accuracy: 0.7917 - val_loss: 0.4616 - val_accuracy: 0.7996
0-conv---64-nodes---1-dense---1595919881
Epoch 1/2
  2/529 [..............................] - ETA: 15:05 - loss: 1.2220 - accuracy: 0.4444WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.875274). Check your callbacks.
  3/529 [..............................] - ETA: 11:07 - loss: 1.5099 - accuracy: 0.5185WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.977435). Check your callbacks.
  4/529 [..............................] - ETA: 8:57 - loss: 8.1165 - accuracy: 0.4167 WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.489217). Check your callbacks.
529/529 [==============================] - 157s 297ms/step - loss: 1.1844 - accuracy: 0.7244 - val_loss: 0.5493 - val_accuracy: 0.7580
Epoch 2/2
529/529 [==============================] - 154s 292ms/step - loss: 0.3973 - accuracy: 0.8343 - val_loss: 0.5036 - val_accuracy: 0.7902
0-conv---64-nodes---2-dense---1595920347
Epoch 1/2
  2/529 [..............................] - ETA: 12:34 - loss: 2.9759 - accuracy: 0.1667 WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.231208). Check your callbacks.
529/529 [==============================] - 266s 503ms/step - loss: 0.6934 - accuracy: 0.7271 - val_loss: 0.5445 - val_accuracy: 0.7826
Epoch 2/2
529/529 [==============================] - 261s 493ms/step - loss: 0.4769 - accuracy: 0.8001 - val_loss: 0.5115 - val_accuracy: 0.7807
0-conv---64-nodes---3-dense---1595921035
Epoch 1/2
  2/529 [..............................] - ETA: 10:17 - loss: 1.5227 - accuracy: 0.2778WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.054166). Check your callbacks.
529/529 [==============================] - 293s 554ms/step - loss: 0.7026 - accuracy: 0.7042 - val_loss: 0.6996 - val_accuracy: 0.6975
Epoch 2/2
529/529 [==============================] - 281s 532ms/step - loss: 0.5167 - accuracy: 0.7803 - val_loss: 0.4647 - val_accuracy: 0.7921
0-conv---128-nodes---1-dense---1595921841
Epoch 1/2
  2/529 [..............................] - ETA: 16:24 - loss: 8.4130 - accuracy: 0.3333 WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.664864). Check your callbacks.
529/529 [==============================] - 323s 611ms/step - loss: 1.6026 - accuracy: 0.7194 - val_loss: 0.5205 - val_accuracy: 0.7902
Epoch 2/2
529/529 [==============================] - 313s 592ms/step - loss: 0.4310 - accuracy: 0.8226 - val_loss: 0.5366 - val_accuracy: 0.7750
0-conv---128-nodes---2-dense---1595922863
Epoch 1/2
  2/529 [..............................] - ETA: 12:51 - loss: 3.6535 - accuracy: 0.2222WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (1.211521). Check your callbacks.
529/529 [==============================] - 568s 1s/step - loss: 0.7377 - accuracy: 0.7030 - val_loss: 0.5345 - val_accuracy: 0.7694
Epoch 2/2
529/529 [==============================] - 550s 1s/step - loss: 0.4739 - accuracy: 0.8016 - val_loss: 0.5118 - val_accuracy: 0.7694
0-conv---128-nodes---3-dense---1595924202
Epoch 1/2
  2/529 [..............................] - ETA: 31:55 - loss: 2.1914 - accuracy: 0.2778WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (3.035765). Check your callbacks.
  3/529 [..............................] - ETA: 27:51 - loss: 1.8927 - accuracy: 0.2963WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (2.244955). Check your callbacks.
 56/529 [==>...........................] - ETA: 13:38 - loss: 1.0876 - accuracy: 0.5258
 '''

'''
0-coonv---32-nodes---1-dense---1595918749
529/529 [==============================] - 77s 145ms/step - loss: 0.3876 - accuracy: 0.8400 - val_loss: 0.5005 - val_accuracy: 0.7788

0-coonv---32-nodes---2-dense---1595918911
529/529 [==============================] - 127s 240ms/step - loss: 0.4883 - accuracy: 0.7965 - val_loss: 0.5098 - val_accuracy: 0.7750

0-coonv---32-nodes---3-dense---1595919185
529/529 [==============================] - 138s 262ms/step - loss: 0.4884 - accuracy: 0.7917 - val_loss: 0.4616 - val_accuracy: 0.7996

0-coonv---64-nodes---1-dense---1595919881
529/529 [==============================] - 154s 292ms/step - loss: 0.3973 - accuracy: 0.8343 - val_loss: 0.5036 - val_accuracy: 0.7902

0-coonv---64-nodes---2-dense---1595920347
529/529 [==============================] - 261s 493ms/step - loss: 0.4769 - accuracy: 0.8001 - val_loss: 0.5115 - val_accuracy: 0.7807

0-coonv---64-nodes---3-dense---1595921035
529/529 [==============================] - 281s 532ms/step - loss: 0.5167 - accuracy: 0.7803 - val_loss: 0.4647 - val_accuracy: 0.7921

0-coonv---128-nodes---1-dense---1595921841
529/529 [==============================] - 313s 592ms/step - loss: 0.4310 - accuracy: 0.8226 - val_loss: 0.5366 - val_accuracy: 0.7750

0-coonv---128-nodes---2-dense---1595922863
529/529 [==============================] - 550s 1s/step - loss: 0.4739 - accuracy: 0.8016 - val_loss: 0.5118 - val_accuracy: 0.7694

0-coonv---128-nodes---3-dense---1595924202
Epoch 1/2
  2/529 [..............................] - ETA: 31:55 - loss: 2.1914 - accuracy: 0.2778WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (3.035765). Check your callbacks.
  3/529 [..............................] - ETA: 27:51 - loss: 1.8927 - accuracy: 0.2963WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (2.244955). Check your callbacks.
 56/529 [==>...........................] - ETA: 13:38 - loss: 1.0876 - accuracy: 0.5258
'''

# Koriscenje modela

In [ ]:
model = tf.keras.models.load_model('saved_model\\100')# kod za ucitavanje modela koji se nalazi u folderu saved_model koji sadrzi folder 0-dense---32-nodes---1-conv---accuracy-95_08

In [ ]:
model.summary()# da proverimo da li je model dobro ucitan/sacuvan

In [ ]:
IMG_LOCATION = 'C:/Users/User/Desktop/xray.jpg'

In [ ]:
test_slika = cv2.imread(IMG_LOCATION, cv2.IMREAD_GRAYSCALE)
test_slika = cv2.resize(test_slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
plt.imshow(test_slika, cmap='gray')

test_slika = np.array(test_slika)
test_slika = (np.expand_dims(test_slika, 0))
test_slika = (np.expand_dims(test_slika,-1))

prediction = model.predict(test_slika)
print("Model je misli da je u pitanju {}".format(KATEGORIJE[np.argmax(prediction[0])]))

In [15]:
csv_fajl = pandas.read_csv(PUTANJA + r"\chest_xray_metadata.csv")  # promeniti na svoju lokaciju

# uklanjamo pusace zbog manjka slika (takodje je tako navedeno u zadatku)

train_y = np.ndarray(0)
ssi = 0
for index, row in csv_fajl.iterrows():
    if ssi < 11:
        if row['Label'] != 'Normal':
            if row['Label_1_Virus_category'] == 'Virus':
                train_y = np.append(train_y, 1)
            elif row['Label_1_Virus_category'] == 'bacteria':
                train_y = np.append(train_y, 2)
            else:
                print("Neodgovarajuca labela na slici {}".format(row['X_ray_image_name']))
                train_y = np.append(train_y, -1)
        else:
            train_y = np.append(train_y, 0)
        
csv_fajl.head()
training_data = []
gresaka = 0
uspeha = 0
for index, row in csv_fajl.iterrows():
    if train_y[index] != -1:
        img_name = row['X_ray_image_name']
        lokacija = PUTANJA + '/snimci/' + str(img_name)
        test_slika = cv2.imread(lokacija, cv2.IMREAD_GRAYSCALE)
        test_slika = cv2.resize(test_slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
        test_slika = np.array(test_slika)
        test_slika = (np.expand_dims(test_slika, 0))
        test_slika = (np.expand_dims(test_slika,-1))

        prediction = model.predict(test_slika)
        if int(np.argmax(prediction[0])) != int(train_y[index]):
            gresaka += 1
        else:
            uspeha += 1

        
print("Pogresio {} puta, a pogodio {}".format(str(gresaka), str(uspeha)))



Neodgovarajuca labela na slici acute-respiratory-distress-syndrome-ards.jpg
Neodgovarajuca labela na slici acute-respiratory-distress-syndrome-ards-1.jpg
Pogresio 336 puta, a pogodio 4948


In [ ]:
print(len(csv_fajl))

# 152 netacnih od 5286 (2.8% greske)

In [16]:
PUTANJA = 'C:/Users/User/Desktop/chest-xray-dataset-test'
csv_fajl = pandas.read_csv(PUTANJA + r"\chest_xray_test_dataset.csv")
print(len(csv_fajl))

train_y = np.ndarray(0)
ssi = 0
for index, row in csv_fajl.iterrows():
    if ssi < 11:
        if row['Label'] != 'Normal':
            if row['Label_1_Virus_category'] == 'Virus':
                train_y = np.append(train_y, 1)
            elif row['Label_1_Virus_category'] == 'bacteria':
                train_y = np.append(train_y, 2)
            else:
                print("Neodgovarajuca labela na slici {}".format(row['X_ray_image_name']))
                train_y = np.append(train_y, -1)
        else:
            train_y = np.append(train_y, 0)
        
csv_fajl.head()
training_data = []
gresaka = 0
uspeha = 0
for index, row in csv_fajl.iterrows():
    if train_y[index] != -1:
        img_name = row['X_ray_image_name']
        lokacija = PUTANJA + '/test/' + str(img_name)
        test_slika = cv2.imread(lokacija, cv2.IMREAD_GRAYSCALE)
        test_slika = cv2.resize(test_slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
        test_slika = np.array(test_slika)
        test_slika = (np.expand_dims(test_slika, 0))
        test_slika = (np.expand_dims(test_slika,-1))

        prediction = model.predict(test_slika)
        if int(np.argmax(prediction[0])) != int(train_y[index]):
            gresaka += 1
        else:
            uspeha += 1

        
print("Pogresio {} puta, a pogodio {}".format(str(gresaka), str(uspeha)))
'''
train_y = np.ndarray(0)
ssi = 0
for index, row in csv_fajl.iterrows():
    if ssi < 11:
        if row['Label'] != 'Normal':
            if str(row['Label_2_Virus_category']) != 'nan':
                if row['Label_2_Virus_category'] == 'Streptococcus':
                    #csv_fajl.at[str(index), 'Label'] = 3
                    train_y = np.append(train_y, 3)
                    #row['Label'] = 3
                elif row['Label_2_Virus_category'] == 'COVID-19':
                    #csv_fajl.at[str(index), 'Label'] = 4
                    train_y = np.append(train_y, 4)
                    #row['Label'] = 4
                elif row['Label_2_Virus_category'] == 'ARDS':
                    #csv_fajl.at[str(index), 'Label'] = 5
                    train_y = np.append(train_y, 5)
                    #row['Label'] = 5
                elif row['Label_2_Virus_category'] == 'SARS':
                    #csv_fajl.at[str(index), 'Label'] = 6
                    train_y = np.append(train_y, 6)
                    #row['Label'] = 6
                else:
                    print('promaseno ' + row['Label_2_Virus_category'])
                # print(row['Label'], " => " , row['Label_1_Virus_category'], "|", row['Label_2_Virus_category'])#, row['X_ray_image_name'])
            else:
                if row['Label_1_Virus_category'] == 'Virus':
                    #csv_fajl.at[str(index), 'Label'] = 1
                    train_y = np.append(train_y, 1)
                    #row['Label'] = 1
                elif row['Label_1_Virus_category'] == 'bacteria':
                    #csv_fajl.at[str(index), 'Label'] = 2
                    train_y = np.append(train_y, 2)
                    #row['Label'] = 2
                else:
                    print('promaseno ' + row['Label_1_Virus_category'])
                # print(row['Label'], " => " , row['Label_1_Virus_category'], "|", row['Label_2_Virus_category'])#, row['X_ray_image_name'])
        else:
            #csv_fajl.at[str(index), 'Label'] = 0
            train_y = np.append(train_y, 0)
            #row['Label'] = 0
        
csv_fajl.head()
training_data = []
print(ssi)
ssi3 = 0
ssi2 = 0;

for s in train_y:
    print(s)
print("-----------")

for index, row in csv_fajl.iterrows():
    img_name = row['X_ray_image_name']
    lokacija = PUTANJA + '/test/' + str(img_name)
    test_slika = cv2.imread(lokacija, cv2.IMREAD_GRAYSCALE)
    test_slika = cv2.resize(test_slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    test_slika = np.array(test_slika)
    test_slika = (np.expand_dims(test_slika, 0))
    test_slika = (np.expand_dims(test_slika,-1))

    prediction = model.predict(test_slika)
    
    if int(np.argmax(prediction[0])) != int(train_y[index]):
        print(prediction[0])
        print("{} == {}".format(KATEGORIJE[int(train_y[index])], KATEGORIJE[np.argmax(prediction[0])]))
        ssi3 += 1
    else:
        ssi2 += 1
        print(str(ssi2) + '--' + str(ssi3))

        
print(str(ssi2) + ' tacnih odgovora, a ' + str(ssi3) + ' netacnih odgovora')
'''

624
Pogresio 257 puta, a pogodio 367


'\ntrain_y = np.ndarray(0)\nssi = 0\nfor index, row in csv_fajl.iterrows():\n    if ssi < 11:\n        if row[\'Label\'] != \'Normal\':\n            if str(row[\'Label_2_Virus_category\']) != \'nan\':\n                if row[\'Label_2_Virus_category\'] == \'Streptococcus\':\n                    #csv_fajl.at[str(index), \'Label\'] = 3\n                    train_y = np.append(train_y, 3)\n                    #row[\'Label\'] = 3\n                elif row[\'Label_2_Virus_category\'] == \'COVID-19\':\n                    #csv_fajl.at[str(index), \'Label\'] = 4\n                    train_y = np.append(train_y, 4)\n                    #row[\'Label\'] = 4\n                elif row[\'Label_2_Virus_category\'] == \'ARDS\':\n                    #csv_fajl.at[str(index), \'Label\'] = 5\n                    train_y = np.append(train_y, 5)\n                    #row[\'Label\'] = 5\n                elif row[\'Label_2_Virus_category\'] == \'SARS\':\n                    #csv_fajl.at[str(index), \'Label\'

# 205 netacnih od 624 (32% greske)

In [17]:
PUTANJA = 'C:/Users/User/Desktop/chest-xray-dataset-test'
model = tf.keras.models.load_model('saved_model\\100')



csv_fajl = pandas.read_csv(PUTANJA + r"\chest_xray_test_dataset.csv")

train_y = np.ndarray(0)
ssi = 0
for index, row in csv_fajl.iterrows():
    if ssi < 11:
        if row['Label'] != 'Normal':
            if str(row['Label_2_Virus_category']) != 'nan':
                if row['Label_2_Virus_category'] == 'Streptococcus':
                    #csv_fajl.at[str(index), 'Label'] = 3
                    train_y = np.append(train_y, 3)
                    #row['Label'] = 3
                elif row['Label_2_Virus_category'] == 'COVID-19':
                    #csv_fajl.at[str(index), 'Label'] = 4
                    train_y = np.append(train_y, 4)
                    #row['Label'] = 4
                elif row['Label_2_Virus_category'] == 'ARDS':
                    #csv_fajl.at[str(index), 'Label'] = 5
                    train_y = np.append(train_y, 5)
                    #row['Label'] = 5
                elif row['Label_2_Virus_category'] == 'SARS':
                    #csv_fajl.at[str(index), 'Label'] = 6
                    train_y = np.append(train_y, 6)
                    #row['Label'] = 6
                else:
                    pass
                # print(row['Label'], " => " , row['Label_1_Virus_category'], "|", row['Label_2_Virus_category'])#, row['X_ray_image_name'])
            else:
                if row['Label_1_Virus_category'] == 'Virus':
                    #csv_fajl.at[str(index), 'Label'] = 1
                    train_y = np.append(train_y, 1)
                    #row['Label'] = 1
                elif row['Label_1_Virus_category'] == 'bacteria':
                    #csv_fajl.at[str(index), 'Label'] = 2
                    train_y = np.append(train_y, 2)
                    #row['Label'] = 2
                else:
                    pass
                # print(row['Label'], " => " , row['Label_1_Virus_category'], "|", row['Label_2_Virus_category'])#, row['X_ray_image_name'])
        else:
            #csv_fajl.at[str(index), 'Label'] = 0
            train_y = np.append(train_y, 0)
            #row['Label'] = 0
        
csv_fajl.head()
training_data = []
ssi3 = 0
ssi2 = 0;

print("Ucitao " + str(len(train_y)) + " objekata")

for index, row in csv_fajl.iterrows():
    img_name = row['X_ray_image_name']
    lokacija = PUTANJA + '/test/' + str(img_name)
    test_slika = cv2.imread(lokacija, cv2.IMREAD_GRAYSCALE)
    test_slika = cv2.resize(test_slika, dsize=(IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_CUBIC)
    test_slika = np.array(test_slika)
    test_slika = (np.expand_dims(test_slika, 0))
    test_slika = (np.expand_dims(test_slika,-1))

    prediction = model.predict(test_slika)
    
    if int(np.argmax(prediction[0])) != int(train_y[index]):
        ssi3 += 1
    else:
        ssi2 += 1

        
print(str(ssi2) + ' tacnih odgovora, a ' + str(ssi3) + ' netacnih odgovora (' + str(round((ssi2)/(len(train_y))*100, 1)) + "% tacnosti)")


Ucitao 624 objekata
419 tacnih odgovora, a 205 netacnih odgovora (67.1% tacnosti)
